# Using RNN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import pydotplus as pyd

import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, concatenate

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
%matplotlib inline
import os
import re
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Pre-Processing


In [ ]:
df1 = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
df = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()
df3=pd.concat([df1,df])
df3
testing=df1['labels']
testing

0       1
1       0
2       0
3       1
4       1
       ..
1795    1
1796    0
1797    0
1798    1
1799    1
Name: labels, Length: 1800, dtype: int64

- ### Filtering headlines 


In [ ]:
# df['response'] = df['response'].apply(lambda x: x.lower())
# df['response'] = df['response'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


# df1['response'] = df1['response'].apply(lambda x: x.lower())
# df1['response'] = df1['response'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

- ### Defining Max features 
- ### using Tokenizer to vectorize and convert text into Sequences 
 so that the Network can deal with it as input

In [ ]:
for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df3['response'].values)
X_train = tokenizer.texts_to_sequences(df['response'].values)
X_train = pad_sequences(X_train,100)

X_test = tokenizer.texts_to_sequences(df1['response'].values)
X_test = pad_sequences(X_test,100)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

16064


In [ ]:
for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures1 = 5000
tokenizer1 = Tokenizer(num_words=max_fatures1, split=' ')
tokenizer1.fit_on_texts(df3['context'].values)
X_train1 = tokenizer1.texts_to_sequences(df['context'].values)
X_train1 = pad_sequences(X_train,100)

X_test1 = tokenizer1.texts_to_sequences(df1['context'].values)
X_test1 = pad_sequences(X_test,100)

vocab_size1 = len(tokenizer1.word_index)
print(vocab_size1)

22758


### Splitting data to train & test

In [ ]:
Y_train, Y_test = df['labels'],df1['labels']

Y_train, Y_test = pd.get_dummies(Y_train).values, pd.get_dummies(Y_test).values

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5000, 100) (5000, 2)
(1800, 100) (1800, 2)


In [ ]:
Y_train, Y_test = df['labels'],df1['labels']

Y_train, Y_test = pd.get_dummies(Y_train).values, pd.get_dummies(Y_test).values

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(X_train1.shape,Y_train.shape)
print(X_test1.shape,Y_test.shape)

(5000, 100) (5000, 2)
(1800, 100) (1800, 2)
(5000, 100) (5000, 2)
(1800, 100) (1800, 2)


# Defining the LSTM RNN Model
With softmax activation

In [ ]:
embed_dim = 128
lstm_out = 196

model11 = Sequential()
model11.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model11.add(SpatialDropout1D(0.4))
model11.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model11.add(Dense(2,activation='softmax'))
model11.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
# import tensorflow as tf
# inp1 = Input(shape=(100,))
# x=Embedding(max_fatures, embed_dim,input_length = X_train.shape[1])(inp1)
# x=SpatialDropout1D(0.4)(x)
# # x = Embedding(vocab_size, 200, weights=[embedding_matrix])(inp1)
# x = tf.keras.layers.LSTM(256, dropout=0.2, recurrent_dropout=0.2)(x)
# # x = tf.keras.layers.Bidirectional(LSTM(150))(x)
# # x = Dense(128, activation="relu")(x)



In [ ]:

# inp2 = Input(shape=(100,))
# x1=Embedding(max_fatures, embed_dim,input_length = X_train.shape[1])(inp2)
# x1=SpatialDropout1D(0.4)(x1)
# # x = Embedding(vocab_size, 200, weights=[embedding_matrix])(inp1)
# x1 = tf.keras.layers.LSTM(256, dropout=0.2, recurrent_dropout=0.2)(x1)
# # x1= tf.keras.layers.Bidirectional(LSTM(150))(x1)
# # x1 = Dense(128, activation="relu")(x1)


In [ ]:
# concatted = tf.keras.layers.Concatenate()([x, x1])

# # concatted = Dropout(0.1)(concatted)
# concatted = Dense(64, activation="relu")(concatted)
# concatted = Dense(2, activation="softmax")(concatted)    
# model21 = Model(inputs=[inp1,inp2], outputs=concatted)

# print(model21.summary())
# model21.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              mode = 'max', 
                              factor=0.9, 
                              patience=1, 
                              min_lr=0.00001, 
                              verbose=10)

checkpoint = ModelCheckpoint("models/sentiment_model.h5", 
                               monitor="val_accuracy", 
                               mode="max", 
                               save_best_only = True, 
                               verbose=1)

earlystop = EarlyStopping(monitor = 'val_accuracy', 
                            mode="max", 
                            min_delta = 0, 
                            patience = 4,
                            verbose=1)

In [ ]:
X_test_11=X_test

In [ ]:
batch_size = 128
# history=model21.fit([X_train,X_train1] ,Y_train, epochs = 25, batch_size=batch_size,validation_data=([X_test,X_test1],Y_test), callbacks=[ earlystop, reduce_lr])
history=model11.fit(X_train ,Y_train, epochs = 3, batch_size=batch_size,validation_data=(X_test,Y_test), callbacks=[ earlystop, reduce_lr])

Epoch 1/3
40/40 [==============================] - 25s 566ms/step - loss: 0.6702 - accuracy: 0.5786 - val_loss: 0.6795 - val_accuracy: 0.6067
Epoch 2/3
40/40 [==============================] - 23s 568ms/step - loss: 0.5137 - accuracy: 0.7434 - val_loss: 0.7258 - val_accuracy: 0.6289
Epoch 3/3
40/40 [==============================] - 22s 552ms/step - loss: 0.4265 - accuracy: 0.8049 - val_loss: 0.6734 - val_accuracy: 0.6172

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.


Let's check the accuracy and score now

### A graph of accuracy & Loss vs Epoch

# Using CNN

In [ ]:
import pandas as pd
import numpy as np
from numpy import asarray
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import pydotplus as pyd

import keras
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input, Flatten, Embedding, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, concatenate

In [ ]:
df1 = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
df = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796,1
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797,0
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798,0
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799,1
1799,NOT_SARCASM,[Not long wrapped on the amazing #January22nd ...,@USER @USER @USER you still need to send the l...,twitter_1800,1


In [ ]:
df["tweets"]=df['response']
df1["tweets"]=df1['response']
data=df
df1

,label,context,response,id,labels,tweets
0,NOT_SARCASM,"[Well now that ’ s problematic AF <URL>, @USER...","@USER @USER @USER My 3 year old , that just fi...",twitter_1,1,"@USER @USER @USER My 3 year old , that just fi..."
1,SARCASM,[Last week the Fake News said that a section o...,@USER @USER How many verifiable lies has he to...,twitter_2,0,@USER @USER How many verifiable lies has he to...
2,SARCASM,[@USER Let ’ s Aplaud Brett When he deserves i...,@USER @USER @USER Maybe Docs just a scrub of a...,twitter_3,0,@USER @USER @USER Maybe Docs just a scrub of a...
3,NOT_SARCASM,[Women generally hate this president . What's ...,@USER @USER is just a cover up for the real ha...,twitter_4,1,@USER @USER is just a cover up for the real ha...
4,NOT_SARCASM,"[Dear media Remoaners , you excitedly sharing ...",@USER @USER @USER The irony being that he even...,twitter_5,1,@USER @USER @USER The irony being that he even...
...,...,...,...,...,...,...
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796,1,@USER @USER @USER is definitely the best out t...
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797,0,@USER @USER Ye let her out run wild and infect...
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798,0,"@USER @USER @USER Thanks for that , I would ha..."
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799,1,@USER @USER @USER Yes also #found this on #new...


In [ ]:
X_s = data['tweets'].values
y_s = data['labels'].values

Xt_s = df1['tweets'].values
yt_s = df1['labels'].values
# X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = X_s,Xt_s,y_s,yt_s

In [ ]:
t = Tokenizer(filters='"$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
t.fit_on_texts(X_train)
vocab_size = len(t.word_index) + 1
print(vocab_size)

14017


In [ ]:
encoded_train = t.texts_to_sequences(X_train)
encoded_test = t.texts_to_sequences(X_test)

In [ ]:
max_length = 140
# pad_len=max(max([len(X_train[i]) for i in range(len(X_train))]),max([len(X_test[i]) for i in range(len(X_test))])) #tunable
padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/MyDrive/sarcasm-detection/embeddings/glove.twitter.27B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 1193514 word vectors.


In [ ]:

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 200))
for word, i in t.word_index.items():
    try:
        embedding_vector = embeddings_index[word]
    except:
        embedding_vector = [0]*200
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
embedding_matrix.shape

(14017, 200)

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
y_train.shape
print(y_train)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
input_data = Input(shape=(max_length,), name='main_input')
embedding_layer = Embedding(vocab_size, 200, weights=[embedding_matrix], trainable=False)(input_data)
conv_1 = Conv1D(filters=50, kernel_size=4, activation='relu')(embedding_layer)
max_1 = MaxPooling1D(pool_size=2)(conv_1)
conv_2 = Conv1D(filters=100, kernel_size=3, activation='relu')(max_1)
max_2 = MaxPooling1D(pool_size=2)(conv_2)
# layer=LSTM(128, kernel_initializer='he_normal', activation='sigmoid')(max_2)
flatten = Flatten()(max_2)
dense = Dense(100, activation='relu', name='fully_connected')(flatten)
out = Dense(2, activation='softmax')(dense)

model_01 = Model(inputs=[input_data], outputs=[out])

print(model_01.summary())

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 140)]             0         
_________________________________________________________________
embedding_25 (Embedding)     (None, 140, 200)          2803400   
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 137, 50)           40050     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 68, 50)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 66, 100)           15100     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 33, 100)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3300)              0  

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_f1_m', 
                              mode = 'max', 
                              factor=0.5, 
                              patience=5, 
                              min_lr=0.0001, 
                              verbose=10)

checkpoint = ModelCheckpoint("models/sentiment_model.h5", 
                               monitor="val_f1_m", 
                               mode="max", 
                               save_best_only = True, 
                               verbose=1)

earlystop = EarlyStopping(monitor = 'val_f1_m', 
                            mode="max", 
                            min_delta = 0, 
                            patience = 5,
                            verbose=1)

In [ ]:
c = optimizers.Adam(lr = 0.00005)
model_01.compile(optimizer=c, loss='categorical_crossentropy', metrics=[f1_m, 'acc'])

h1 = model_01.fit(padded_train, y_train, 
               batch_size=64, 
               epochs=50, 
               verbose=1, callbacks=[checkpoint, earlystop, reduce_lr],
               validation_data=(padded_test, y_test)) 
              #  validation_split=0.2)

Epoch 1/50
79/79 [==============================] - 3s 25ms/step - loss: 0.6757 - f1_m: 0.5752 - acc: 0.5752 - val_loss: 0.7044 - val_f1_m: 0.5356 - val_acc: 0.5289

Epoch 00001: val_f1_m improved from -inf to 0.53556, saving model to models/sentiment_model.h5
Epoch 2/50
79/79 [==============================] - 2s 21ms/step - loss: 0.6385 - f1_m: 0.6334 - acc: 0.6333 - val_loss: 0.6879 - val_f1_m: 0.5506 - val_acc: 0.5444

Epoch 00002: val_f1_m improved from 0.53556 to 0.55065, saving model to models/sentiment_model.h5
Epoch 3/50
79/79 [==============================] - 2s 21ms/step - loss: 0.6107 - f1_m: 0.6645 - acc: 0.6644 - val_loss: 0.6755 - val_f1_m: 0.5803 - val_acc: 0.5750

Epoch 00003: val_f1_m improved from 0.55065 to 0.58028, saving model to models/sentiment_model.h5
Epoch 4/50
79/79 [==============================] - 2s 21ms/step - loss: 0.5672 - f1_m: 0.7152 - acc: 0.7152 - val_loss: 0.6663 - val_f1_m: 0.6105 - val_acc: 0.6061

Epoch 00004: val_f1_m improved from 0.58028 t

In [ ]:
X_test_01=padded_test

#SVM

In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
import demoji
demoji.download_codes()
nltk.download('stopwords') 
nltk.download('wordnet')

... OK (Got response in 0.39 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
#df1['response'] = df1['response'].apply(preprocess_text)

In [ ]:
df['context_resp']=df['label']
for i in range(5000):
  df['context_resp'][i]=''.join(df['context'][i]) + df['response'][i]

print(df['context'][12])
print(df['response'][12])
print(df['context_resp'][12])

['Matt here . Rarely comment on the president , but having someone who is not Jewish tell Jews what they should or shouldn ’ t think is beyond the pale and it should not go unchallenged . <URL>', '@USER It seems there is a long history of non-Jews telling Jews what to think .']
@USER @USER Hey , but what do they have to lose ? Asking for some black friends .
Matt here . Rarely comment on the president , but having someone who is not Jewish tell Jews what they should or shouldn ’ t think is beyond the pale and it should not go unchallenged . <URL>@USER It seems there is a long history of non-Jews telling Jews what to think .@USER @USER Hey , but what do they have to lose ? Asking for some black friends .


In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@USER","", text)
    text = re.sub(r'<URL>','',text)
    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    ps=PorterStemmer()
    keywords=[ps.stem(x) for x in keywords]
    return keywords

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
for i in range(5):
  print(df['response'][i])
  print(preprocess_text(df['response'][i]))
  print(preprocess_text2(df['response'][i]))
  print()
  print(df['context_resp'][i])
  print(preprocess_text(df['context_resp'][i]))
  print(preprocess_text2(df['context_resp'][i]))
  print()

@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
['get', 'obvious', 'care', 'would', 'move', 'right', 'along', 'instead', 'decid', 'care', 'troll']
['get', '..', 'obvious', 'care', "would'v", 'move', 'right', 'along', '..', 'instead', 'decid', 'care', 'troll', '..']

A minor child deserves privacy and should be kept out of politics . Pamela Karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it .@USER If your child isn't named Barron ... #BeBest Melania couldn't care less . Fact . 💯@USER @USER @USER I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..
['minor', 'child', 'deserv', 'privaci', 'keep', 'polit', 'pamela', 'karlan', 'asham', 'angri', 'obvious', 'bia', 'public', 'pander', 'use', 'child', 'child', 'name', 'barron', 'bebest', 'melania', 'care', 'less', 'fa

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(df['label'].value_counts())
print(df1['label'].value_counts())

SARCASM        2500
NOT_SARCASM    2500
Name: label, dtype: int64
SARCASM        900
NOT_SARCASM    900
Name: label, dtype: int64


In [ ]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(df['response'],df['label'],random_state=123)
X_test_svm, Y_test = df1['response'],df1['label']

In [ ]:
pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.2.0)


In [ ]:
from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sbert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
# sgd_clf = SGDClassifier(random_state=123)
from tqdm.auto import tqdm
import numpy as np
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
sentence_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_train)]

In [ ]:
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
test_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_test_svm)]

In [ ]:
sentence_embeddings=np.array(sentence_embeddings)
test_embeddings=np.array(test_embeddings)

In [ ]:
# X_train2, X_test2, Y_train2, Y_test2=train_test_split(sentence_embeddings,Y_train,random_state=42)
X_train2, X_test2, Y_train2, Y_test2=sentence_embeddings, test_embeddings, Y_train, Y_test

In [ ]:
X_test_=test_embeddings

In [ ]:
from sklearn.svm import SVC
model= SVC(random_state=24,probability=True)

model.fit(sentence_embeddings, Y_train) # fit the model
y_pred= model.predict(test_embeddings) # then predict on the test set
accuracy= accuracy_score(Y_test, y_pred) # this gives us how often the algorithm predicted correctly
# clf_report= classification_report(Y_test, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
# print(clf_report)
print("\n")

The accuracy of model SVC is 0.70




In [ ]:
pr=model.predict_proba(test_embeddings)
pr

array([[0.24555769, 0.75444231],
       [0.24784168, 0.75215832],
       [0.32731868, 0.67268132],
       ...,
       [0.19164691, 0.80835309],
       [0.89704209, 0.10295791],
       [0.97572804, 0.02427196]])

In [ ]:
pr2=np.subtract(pr,1)
pr3=np.multiply(pr2,-1)
pr3

array([[0.75444231, 0.24555769],
       [0.75215832, 0.24784168],
       [0.67268132, 0.32731868],
       ...,
       [0.80835309, 0.19164691],
       [0.10295791, 0.89704209],
       [0.02427196, 0.97572804]])

# SVC_tfidf


In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords') 
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm-detection/Datasets/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)
#df1['response'] = df1['response'].apply(preprocess_text)

In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    text = re.sub(r"@USER","", text)
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(df['label'].value_counts())
print(df1['label'].value_counts())

SARCASM        2500
NOT_SARCASM    2500
Name: label, dtype: int64
SARCASM        900
NOT_SARCASM    900
Name: label, dtype: int64


In [ ]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(df['response'],df['label'],random_state=123)
X_test, Y_test = df1['response'],df1['label']

In [ ]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train_tfidf = vectoriser.fit_transform(X_train)
X_train_tfidf.shape

(5000, 10703)

In [ ]:
sgd_clf = SGDClassifier(random_state=123)
sgf_clf_scores = cross_val_score(sgd_clf, X_train_tfidf, Y_train, cv=5)
print(sgf_clf_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (sgf_clf_scores.mean(), sgf_clf_scores.std() * 2))

[0.726 0.722 0.716 0.709 0.704]
Accuracy: 0.72 (+/- 0.02)


In [ ]:
sgf_clf_pred = cross_val_predict(sgd_clf, X_train_tfidf, Y_train, cv=5)
print(confusion_matrix(Y_train, sgf_clf_pred))

[[1761  739]
 [ 684 1816]]


In [ ]:
grid = {'fit_intercept': [True,False],
        'early_stopping': [True, False],
        'loss' : ['hinge', 'log', 'squared_hinge'],
        'penalty' : ['l2', 'l1', 'none']}
search = GridSearchCV(estimator=sgd_clf, param_grid=grid, cv=5)
search.fit(X_train_tfidf, Y_train)
search.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


{'early_stopping': False,
 'fit_intercept': True,
 'loss': 'log',
 'penalty': 'l2'}

In [ ]:
grid_sgd_clf_scores = cross_val_score(search.best_estimator_, X_train_tfidf, Y_train, cv=5)
print(grid_sgd_clf_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (grid_sgd_clf_scores.mean(), grid_sgd_clf_scores.std() * 2))

[0.737 0.733 0.732 0.731 0.729]
Accuracy: 0.73 (+/- 0.01)


In [ ]:
pipe = Pipeline([('vectoriser', vectoriser),
                 ('classifier', search.best_estimator_)])
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vectoriser',
                 TfidfVectorizer(analyzer=<function preprocess_text at 0x7f2e3baa44d0>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents...
                ('classifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='log',
                               max_iter=1000, n

In [ ]:
y_test_pred = pipe.predict(X_test)
print("Accuracy: %0.2f" % (accuracy_score(Y_test, y_test_pred)))
print(confusion_matrix(Y_test, y_test_pred))

Accuracy: 0.64
[[553 347]
 [293 607]]


In [ ]:
#all ml models
#!pip install nbsvm-sklearn
#from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24,probability=True)
xboost= XGBClassifier(random_state=24,probability=True)

X_test_tfidf = vectoriser.transform(X_test)
print(X_test_tfidf.shape)
# now, create a list with the objects 
# models= [tree, forest, knn, svm, xboost]
model_svm_tfidf=svm
#models=[nbsvm]
# for model in models:
model_svm_tfidf.fit(X_train_tfidf, Y_train) # fit the model
xboost.fit(X_train_tfidf, Y_train)
tree.fit(X_train_tfidf, Y_train)
forest.fit(X_train_tfidf, Y_train)
knn.fit(X_train_tfidf, Y_train)

(1800, 10703)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_svm= model_svm_tfidf.predict_proba(X_test_tfidf)
y_xboost= xboost.predict_proba(X_test_tfidf)
y_tree= tree.predict_proba(X_test_tfidf)
y_knn= knn.predict_proba(X_test_tfidf)
y_forest= forest.predict_proba(X_test_tfidf)# then predict on the test set
# accuracy= accuracy_score(Y_test, y_pred) # this gives us how often the algorithm predicted correctly
    # clf_report= classification_report(Y_test, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
# print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    # print(clf_report)
    # print("\n")

In [ ]:
pr_svm1=np.subtract(y_svm,1)
pr_svm2=np.multiply(pr_svm1,-1)

pr_knn1=np.subtract(y_knn,1)
pr_knn2=np.multiply(pr_knn1,-1)

pr_forest1=np.subtract(y_forest,1)
pr_forest2=np.multiply(pr_forest1,-1)

pr_tree1=np.subtract(y_tree,1)
pr_tree2=np.multiply(pr_tree1,-1)

pr_xboost1=np.subtract(y_xboost,1)
pr_xboost2=np.multiply(pr_xboost1,-1)

#Ensemble

In [ ]:
# model23.predict(X_test)

In [ ]:
# model_01.predict(X_test)

In [ ]:
members=[model23,model_01,model]


	# make predictions
# yhats = [model11.predict(X_test_11),model_01.predict(X_test_01),pr3,pr_svm2,pr_tree2,pr_forest2,pr_knn2,pr_xboost2]\
yhats = [model11.predict(X_test_11),model_01.predict(X_test_01),pr3,pr_svm2,pr_tree2,pr_forest2,pr_knn2,pr_xboost2]
import numpy as np
yhats = np.array(yhats)
# sum across ensemble members
summed = np.sum(yhats, axis=0)
# argmax across classes
result = np.argmax(summed, axis=1)
# return result
print(result)
print(testing)
# # evaluate a specific number of members in an ensemble
# def evaluate_n_members(members, n_members, testX, testy):
# 	# select a subset of members
# 	subset = members[:n_members]
# 	print(len(subset))
# 	# make prediction
# 	yhat = ensemble_predictions(subset, testX)
# 	# calculate accuracy
accuracy_score(result, testing)

[1 0 0 ... 1 1 1]
0       1
1       0
2       0
3       1
4       1
       ..
1795    1
1796    0
1797    0
1798    1
1799    1
Name: labels, Length: 1800, dtype: int64


0.6538888888888889

In [ ]:
# members=[model23,model_01]


# def ensemble_predictions(members, testX):
# 	# make predictions
# 	yhats = [model.predict(testX) for model in members]
# 	yhats = array(yhats)
# 	# sum across ensemble members
# 	summed = numpy.sum(yhats, axis=0)
# 	# argmax across classes
# 	result = argmax(summed, axis=1)
# 	return result
 
# # evaluate a specific number of members in an ensemble
# def evaluate_n_members(members, n_members, testX, testy):
# 	# select a subset of members
# 	subset = members[:n_members]
# 	print(len(subset))
# 	# make prediction
# 	yhat = ensemble_predictions(subset, testX)
# 	# calculate accuracy
# 	return accuracy_score(testy, yhat)

In [ ]:
# from sklearn.ensemble import VotingClassifier

